# Simulation d'observations de loi multinomiale 

Nous souhaitons simuler des observations d'une loi multinomiale: 

Considérons l'expérience où on extrait $n$ boules de $k$ différentes couleurs d'une urne avec remise. Les boules de la même couleur sont équivalents. Soit $X_i$ la variable qui compte le nombre de balles de couleur $i$ extraites ($i = 1, \cdots, k$), et soit $p_i$ la probabilité d'extraire une boule de couleur $i$. La probabilité de cette loi multinomiale est alors donnée part :

<math> \begin{align}
f(x_1,\ldots,x_k;n,p_1,\ldots,p_k) & {} = \mathbb{P}(X_1 = x_1, \dots, X_k = x_k) \\
& {} = \begin{cases} { \displaystyle {n! \over x_1!\cdots x_k!}p_1^{x_1}\times\cdots\times p_k^{x_k}}, \quad &
\text{quand } \sum_{i=1}^k x_i=n \\  \\
0 & \text{sinon} \end{cases}
\end{align}
</math>


## Simulation avec numpy

In [ ]:
import numpy as np 

In [ ]:
%%time
#Genere 100 observations multinomiale de paramètre n=1 milliard et p = [1./3,1./4,1./6,3./12]
mult = np.random.multinomial(1e9, [1./3,1./4,1./6,3./12], size=100)
print(mult.shape)
mult

In [ ]:
#On retrouve empiriquement les probs théoriques
(mult/1e9)[:4,:]

## Algorithme de simulation : méthode de la fonction inverse

On peut voir une loi multinomiale comme la répétion de n expériences indépendantes à k issues possibles dont on a ensuite rassemblé les issues (compté le nombre de boules pour chaque couleur). On peut donc simuler chacune des expériences indépendamment. 

Pour générer une variable aléatoire discrète prenant pour valeur $1, \cdots, k$ avec proba $p_1, \cdots, p_k$ il suffit d'appliquer la méthode de la fonction inverse (avec inverse généralisée comme on est dans le cas discret). Plus clairement, on génére $U$ une variable aléatoire uniforme sur $[0,1]$ et on regarde dans quel "bins" on tombe, i.e, on calcule :  

$$j = \min \left\{ j' \in \{1,\dots,k\} : \left(\sum_{i=1}^{j'} p_i\right) - U \geq 0 \right\}$$

En répétant cette algorithme n fois et en rassamblant les issus on obtient une observation multinomiale. Nous présentons dans la cellule suivante l'algorithme "multinomial1" quie met en place cette statégie sans finesse. L'algorithme "multinomial2" est une version vectorisée, plus difficile à déchiffrer mais beaucoup plus rapide. Dans tous les cas on ne pourra pas battre les algos de numpy qui sont implémenté directement en C++.

In [ ]:
def multinomial1(n,prob,size=1,seed=42):
    """size = nombre d'observations multinomiale
    seed permet de fixer l'aléatoire
    
    Attention, meme si le vecteur en entré n'est pas trié, 
    la sortie est toujours trié par ordre décroissant"""
    
    np.random.seed(42)
     
    if abs(np.sum(prob)-1)>10e-16:
        return "Probabilités ne somment pas à 1"
    
    prob = np.sort(prob)[::-1] #tri par ordre décroissant, plus rapide
    
    u = np.random.uniform(size=n*size).reshape(size,n)
    output = np.zeros((size,len(prob)))
    for step in range(size):
        for el in u[step,:]:
            #pour chaque u on rejoute +1 pour la bins dans laquelle il tome
            output[step,get_bin(el,prob)] += 1
        
    return output
    
    
def get_bin(u,prob):
    '''Permet d'obtenir la bins dans laquelle tome un element selon un vecteur de proba'''
    cumprob = np.cumsum(prob)
    for i,p in enumerate(cumprob):
        if u<p:
            return i

In [ ]:
%%time
n = int(1e5)
test = multinomial1(n,np.array([1./3,1./4,1./6,3./12]),size=20)
print(test/float(n))

In [ ]:
def multinomial2(n,prob,size=1,seed=42):
    """Genere une loi multinomiale, approche plus vectorielle que la premiere
    size = nombre d'observations multinomiale
    seed permet de fixer l'aléatoire
    
    Attention, meme si le vecteur en entré n'est pas trié, 
    la sortie est toujours trié par ordre décroissant"""
    
    np.random.seed(seed)
     
    if abs(np.sum(prob)-1)>10e-16:
        return "Probabilités ne somment pas à 1"
    
    prob = np.sort(prob)[::-1] 
    
    u = np.random.uniform(size=n*size).reshape(size,n)
    output = np.zeros((size,len(prob)))
    
    cprob = np.cumsum(prob)
    
    for i,p in enumerate(cprob):
        if i==0:
            output[:,i] = (u<cprob[i]).sum(axis=1)
        else : 
            output[:,i] = ((u<cprob[i]) & (u>=cprob[i-1])).sum(axis=1)
   # output = np.concatenate((output[:,0].reshape(-1,1),np.diff(output,axis=1)),axis=1)
        
    return output

In [ ]:
%%time
n = int(1e5)
test = multinomial2(n,np.array([1./3,1./4,1./6,3./12]),size=20)
print((test/float(n))[:4,:])

Beaucoup plus rapide !

# Toy example for CE Method

Cet example correspond à l'example 2.2 de "A Tutorial on the Cross-Entropy Method". On reproduit les mêmes résultats avec $n = 10$, $y = (1,1,1,1,1,0,0,0,0,0)$, $N=50$, $\rho=0.1$ et initialisation avec des Bernouilli uniformes.

In [ ]:
class ToyCrossEntropy(object):
    """Méthode de la cross entropie pour l'exemple jouet introduit dans  
        De Boer, Kroese, Mannor and Rubinstein (2003). 
        A Tutorial on the Cross-Entropy Method. Annals of Operations 
    
    Attributs:
        y: vecteur binaire dont on veut approcher les éléments.
        N: entier, nombre de simulations pour chaque étape
        perc: entier, correspond au percentile fixé dans l'algo   
        seed: entier, pour fixer l'aléatoire.
    """
    
    def __init__(self,y,N,perc=90, seed=42):
        """Initialise la classe ToyCrossEntropy."""
        self.y = y
        self.n = len(y)
        self.N = N
        self.perc = perc
        self.seed = seed
        
    def Bernouilli(self,p):
        '''Permet de simuler une matrice de taille n*len(p) où p=(p_1,...,p_n)
        Chaque colonne est composé de n Bernouilli(p_j) indépendantes'''
        
        return np.array([np.random.binomial(1,prob,size=self.N) for prob in p]).T

    def S(self,x):
        '''retourne le nombre de "match" entre deux vecteurs binaires'''
        
        return len(x) - np.sum(np.abs(x-self.y))

    def update(self,x,score,q):
        '''fonction permettant d'update les probabilités des bernouilli
        Lire l'article pour plus de détails'''
        
        return np.sum((score>=q)*(x))/float(np.sum((score>=q)))

    def CE_fit(self):
        #fixe aléa
        np.random.seed(42)
        
        
        #initialise with Bernouilli(0.5)
        p_init = np.tile(0.5,self.n)
        
        while(self.S(p_init)!=self.n):


            #generate N vectors of bernouilli variables with probabilities p_init
            x = self.Bernouilli(p_init)
            #compute score for each vectors
            score = np.apply_along_axis(lambda x: self.S(x),1,x)
            #calcul du quantile
            q = np.percentile(score,self.perc)
            print([int(q),np.round(p_init,2)])

            #update of probabilities
            p_init = np.apply_along_axis(lambda x, score=score, q=q: self.update(x,score,q),0,x)

        print([q,np.round(p_init,2)])
        
        return "Convergence"

In [ ]:
y=[1,1,1,1,1,0,0,0,0,0]
N=50
toy = ToyCrossEntropy(y,N)
toy.CE_fit()

# TSE problem using cross entropy method 

(Extracted from "A Tutorial on the Cross-Entropy Method")
The travelling salesman problem (TSP) can be formulated as follows. Consider a weighted graph $G$ with $n$ nodes, labelled $1, 2, \cdots, n$. The nodes represent cities, and the edges represent the roads between the cities. Each edge from $i$ to $j$ has weight or cost $c_{i,j}$, representing the length of the road. The problem is to find the shortest tour that visits all the cities exactly once (except the starting city, which is also the terminating city).

Without loss of generality, let us assume that the graph is complete (each nodes are connected to each other), and that some of the weights may be $+\infty$. Let $\chi$ be the set of all possible tours and let $S(x)$ the total length of tour $x \in \chi$. We can represent each tour via a permutation of $(1, \cdots , n)$. For example for $n = 4$, the permutation $(1, 3, 2, 4)$ represents the tour 1→3→2→4→1. Infact, we may as well represent a tour via a permutation $x = (x_1,\cdots,x_n)$ with $x_1 = 1$. From now on we identify a tour with its corresponding permutation, where $x_1 = 1$. We may now formulate the TSP as follows.

$$\min_{x\in \chi} S(x)  = \min_{x\in \chi} \sum_{i=1}^{n-1} c_{x_i,x_{i+1}} + c_{x_n,x_1}  $$

Note that the number of elements $|\chi| = (n-1)!$. We are looking for $x^*$ the solution of the problem.

# Map Monde

In [3]:
import pandas as pd

In [12]:
df = pd.read_csv("worldcities.csv")
df.head()
locations = df.iloc[[1,50,1000],:][['lat', 'lng']]
locationlist = locations.values.tolist()
len(locationlist)

3

In [18]:
import folium

map_osm = folium.Map(location=[48.82, 2.26],zoom_start=2)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point]).add_to(map_osm)
    
folium.PolyLine(locationlist).add_to(map_osm)

map_osm